# Downloads

In [1]:
#%cd /home/yandex/DLW2021/davidhay/
#!pwd
#!ls -l 

In [2]:
#!rm -rf coco
#!mkdir coco
#%cd coco

In [ ]:
#!wget -c http://images.cocodataset.org/zips/train2017.zip
#!unzip train2017.zip
#!rm train2017.zip


In [ ]:
#!wget -c http://images.cocodataset.org/zips/val2017.zip

#!unzip val2017.zip

#!rm val2017.zip


In [3]:

#!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip

#!unzip annotations_trainval2017.zip

#!rm annotations_trainval2017.zip

# Data Loading

## Imports and Vocabulary 

In [1]:

import os
from collections import Counter
import spacy
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from pycocotools.coco import COCO
from PIL import Image
import pickle

spacy_eng = spacy.load("en_core_web_sm")

class Vocabulary:
    def __init__(self,freq_threshold):
        #setting the pre-reserved tokens int to string tokens
        # PAD- padding symbol
        # SOS- Start of Sentence
        # EOS- end of sentence
        # UNK- unknown word (unknown\ below threshold)
        self.itos = {0:"<PAD>",1:"<SOS>",2:"<EOS>",3:"<UNK>"}
        #string to int tokens
        #its reverse dict self.itos
        self.stoi = {v:k for k,v in self.itos.items()}
        self.freq_threshold = freq_threshold
        
    def __len__(self):
      return len(self.itos)
    
    @staticmethod
    def tokenize(text):
        return [token.text.lower() for token in spacy_eng.tokenizer(text)]
    
    def build_vocab(self, sentence_list):
        frequencies = Counter()
        idx = 4
        for index,sentence in enumerate(sentence_list):

            for word in self.tokenize(sentence):
                frequencies[word] += 1
                
                #add the word to the vocab if it reaches minum frequecy threshold
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    if idx > 0 and idx % 1000==0:
                        print(f"Added {idx} words to vocab")
                    idx += 1
            if index>0 and index%1000==0:
                print(f"Iterated {index} sentences")
             

        print(f"Done, added {idx-1} words to vocabulary")
    
    def numericalize(self,text):
        """ For each word in the text corresponding index token for that word form the vocab built as list """
        tokenized_text = self.tokenize(text)
        result = [ self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text ]
        return result

## Dataset custom class

In [2]:
import pickle
class COCODataset(Dataset):
    """
    COCODataset
    """
    def __init__(self,root_dir,annotation_file,transform=None,freq_threshold=5,
                 load_vocab=False, vocab_loc = "vocab.pkl"):
      """
      can use load_vocab to use a previously created vocabulary (time saving feature)
      freq_threshold: words with a count below this number will be marked as <UNK>
      """
      self.root_dir = root_dir
      self.coco = COCO(annotation_file)
      self.transform = transform
      self.cap_max_size = 0
      #Get image and caption colum from the dataframe
      self.imgs = []
      self.captions = []
      for idx,ann in enumerate(self.coco.anns.values()):
        self.imgs.append(self.coco.loadImgs((ann['image_id']))[0]["file_name"])
        self.captions.append(ann['caption'])
        if (idx) % 1000 == 0 and idx>0:
          print(f"Processed {idx} images and captions")
      print("Finished processing images and captions")
      print(f"Got:{len(set(self.imgs))} pictures with {len(self.captions)} captions!")
      
      #Initialize vocabulary and build vocab
      if load_vocab:
        with open(vocab_loc, "rb") as source:
          self.vocab = pickle.load(source)
        print(f"Loaded vocabulary from {vocab_loc}")
      
      else:
        print("Build vocabulary")
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocab(self.captions)
        print("Finished building vocabulary")
        with open(vocab_loc, "wb") as dest:
          pickle.dump(self.vocab, dest)
      
      print(f"Using {len(self.vocab)} words")
    
    def __len__(self):
      return len(self.imgs)
    
    def __getitem__(self,idx):
      caption = self.captions[idx]
      img_name = self.imgs[idx]
      img_location = os.path.join(self.root_dir,img_name)
      img = Image.open(img_location).convert("RGB")
      
      #apply the transfromation to the image
      if self.transform:
          img = self.transform(img)
      
      #numericalize the caption text
      caption_vec = [self.vocab.stoi["<SOS>"]]
      caption_vec.extend(self.vocab.numericalize(caption))
      caption_vec.append(self.vocab.stoi["<EOS>"])
      
      return img, torch.tensor(caption_vec,dtype=torch.long)

## Dataloader creation

In [3]:
# define a transformation to add some noise and variance to our images
transformation = transforms.Compose([transforms.Resize((299,299), Image.NEAREST),
                                     transforms.ToTensor(),
                                     transforms.RandomVerticalFlip(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                      ])

/home/yandex/DLW2021/pelegv/anaconda3/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [38]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

def new_collate(batch):
    (imgs, targets) = zip(*batch)
    imgs = [x.unsqueeze(0) for x in imgs]
    imgs = torch.cat(imgs,dim=0)
    targets_lens = [len(target) for target in targets]
    targets_pad = pad_sequence(targets, batch_first=True, padding_value=0)

    return imgs, targets_pad, targets_lens

In [ ]:

dataset =  COCODataset(
    root_dir = "/home/yandex/DLW2021/davidhay/coco/val2017",
    annotation_file= "/home/yandex/DLW2021/davidhay/coco/annotations/captions_val2017.json",
    transform=transformation,
    freq_threshold=5,
    load_vocab=False
)

In [41]:
print(dataset.vocab.stoi) 

{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'a': 4, '.': 5, 'in': 6, 'of': 7, 'woman': 8, 'the': 9, 'bench': 10, 'with': 11, 'an': 12, 'and': 13, 'toilet': 14, 'office': 15, 'white': 16, 'on': 17, 'two': 18, 'cat': 19, 'between': 20, 'at': 21, 'man': 22, 'is': 23, 'motorcycle': 24, 'parked': 25, 'standing': 26, 'sitting': 27, 'group': 28, 'to': 29, 'their': 30, 'his': 31, 'front': 32, 'motorcycles': 33, 'next': 34, 'kitchen': 35, 'it': 36, 'dog': 37, 'desk': 38, 'black': 39, ',': 40, 'street': 41, 'computer': 42, 'sky': 43, 'person': 44, 'up': 45, 'bathroom': 46, 'has': 47, 'sink': 48, 'are': 49, 'young': 50, 'yellow': 51, 'small': 52, 'picture': 53, 'giraffe': 54, 'women': 55, 'people': 56, 'oranges': 57, 'bowl': 58, 'vase': 59, 'water': 60, 'close': 61, 'open': 62, 'top': 63, 'zebra': 64, 'looking': 65, 'cupcake': 66, 'plane': 67, 'through': 68, 'field': 69, 'train': 70, 'sits': 71, 'truck': 72, 'door': 73, 'car': 74, 'behind': 75, 'bed': 76, 'flying': 77, 'kites': 78, 'room': 7

In [40]:
BATCH_SIZE = 4
NUM_WORKER = 1
#token to represent the padding
pad_idx = dataset.vocab.stoi["<PAD>"]

data_loader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=False,
    collate_fn=new_collate
)

# Models

In [58]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

def get_device(gpus=1):
    if gpus==1:
        return "cuda" if torch.cuda.is_available() else "cpu"
    else:
        if torch.cuda.is_available():
            return f"cuda:{gpus-1}"
        else:
            return "cpu"

device = get_device(1)


class EncoderCNN(nn.Module):
    def __init__(self, output_size, train_CNN=False):
        super(EncoderCNN, self).__init__()
        self.train_CNN = train_CNN
        self.inception = models.inception_v3(pretrained=True, aux_logits=False)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, output_size)
        self.relu = nn.ReLU()

    def forward(self, images):
        '''
        Input: image vector
        Output: features vector
        '''
        features = self.inception(images)
        output = self.relu(features)
        return output

class DecoderRNNV3(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, n_features):
        '''
        L- caption vec length
        B- batch size
        E- embed size
        H- hidden size
        F- number of features from CNN
        '''
        super(DecoderRNNV3, self).__init__()
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.vocab_size = vocab_size
        self.num_layers = 3
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(input_size=embed_size+n_features,
                            hidden_size=hidden_size, num_layers=self.num_layers, batch_first=True)
        self.fc_out = nn.Linear(in_features=hidden_size,
                                out_features=vocab_size)

    def forward(self, features, captions, cap_lengths):
        # cap_lengths - list of the real length of each caption before padding
        assert features.size(0) == captions.size(0)
        # (h_0, c_0) will be initialized to zeros by default
        # embed captions, shape (B, L, E)
        captions_embed = self.embed(captions)
        # features, shape (B, F)
        # features transform shape to (B, L, F)
        features = torch.unsqueeze(features, dim=1)  # (1,2048) -> (1,1,2048)
        # (1,1,2048) -> (1,77, 2048)
        features = features.repeat((1, captions_embed.size(1), 1))
        # combine features + captions to shape (B, L, E+F) (1,77,2048) -> (1,77,2448)
        combined = torch.cat((features, captions_embed), dim=2)
        # create packedSequence that is better for LSTM
        packed = pack_padded_sequence(
            combined, cap_lengths, batch_first=True, enforce_sorted=False)
        # run through the LSTM network and get output of shape (B, L, H)
        lstm_out, _ = self.lstm(packed)
        # unpack so we can use Linear function (works on Tensor not packSeq)
        output_padded, output_lengths = pad_packed_sequence(
            lstm_out, batch_first=True)

        return self.fc_out(output_padded)

    def caption_features(self, features, vocab, vec_len):
        '''
        Vec_len should be the same as is learning. 
        '''
        assert features.size(
            0) == 1, "Caption features doesn't support batches"
        # features: (B,F) -> (1,1,F)
        # w_embed: (1) -> (1,1,E)
        w0 = torch.tensor(vocab.stoi["<SOS>"]).to(device)
        w0 = torch.unsqueeze(w0, 0)
        w0 = torch.unsqueeze(w0, 0)
        w_embed = self.embed(w0)
        features = torch.unsqueeze(features, 1)
        hi = torch.zeros((self.num_layers, 1, self.hidden_size)).to(device)
        ci = torch.zeros((self.num_layers, 1, self.hidden_size)).to(device)
        output = ["<SOS>"]
        for i in range(vec_len):
            combined = torch.cat((features, w_embed), dim=2)
            if i == 0:
                lstm_out, (hi, ci) = self.lstm(combined)
            else:
                lstm_out, (hi, ci) = self.lstm(combined, (hi, ci))
            next_w = torch.argmax(self.fc_out(lstm_out), dim=2)
            output.append(vocab.itos[next_w.item()])
            # lstm_out: (1,1,F)
            # hi, ci: (num_layers, 1, F)
            # next_w: (1,1,vocab_size)
            w_embed = self.embed(next_w)
        return output

    def sample(self, features, max_seq=50, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []
        inputs = features.unsqueeze(1)
        for i in range(max_seq):
            hiddens, states = self.lstm(inputs, states)          # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))            # outputs:  (batch_size, vocab_size)
            _, predicted = outputs.max(1)                        # predicted: (batch_size)
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)                       # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                         # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)                # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids

class CNNtoRNN(nn.Module):
    def __init__(self, features, embed_size, hidden_size, vocab_size, train_CNN=False):
        super(CNNtoRNN, self).__init__()
        self.encoderCNN = EncoderCNN(features, train_CNN).to(device)
        #self.decoderRNN = DecoderRNN(
        #    embed_size, hidden_size, vocab_size).to(device)
        self.decoderRNN = DecoderRNNV3(embed_size, hidden_size, vocab_size, features).to(device)

    def forward(self, images, captions, cap_lengths, show=False):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions, cap_lengths)
        return outputs

    def caption_images(self, image, vocab, max_len=70):
        # # Inference part
        # # Given the image features generate the captions
        # # input shape: (3,x,y) where, x,y: image size
        # # ouput: captions list
        # batch_size = image.size(0)
        # assert batch_size == 1, "Caption 1 image at a time"
        # image_pred = self.encoderCNN(image)

        # # init the hidden and cell states to zeros
        # hidden_state = torch.zeros((1, self.decoderRNN.hidden_size)).to(device)
        # cell_state = torch.zeros((1, self.decoderRNN.hidden_size)).to(device)

        # # starting input is
        # captions = list()
        # outputs = torch.empty(
        #     (batch_size, max_len, self.decoderRNN.vocab_size)).to(device)
        # hidden_state, cell_state = self.decoderRNN.lstm_cell(
        #     image_pred, (hidden_state, cell_state))
        # out = self.decoderRNN.fc_out(self.decoderRNN.dropout(hidden_state))
        # word_embed = self.decoderRNN.embed(torch.tensor(
        #     vocab.stoi["<SOS>"]).to(device)).unsqueeze(0)
        # for t in range(max_len):
        #     # for the first time step the input is the feature vector
        #     # if t == 0:
        #     #     hidden_state, cell_state = self.decoderRNN.lstm_cell(image_pred, (hidden_state, cell_state))
        #     # for the 2nd+ time step, use previously generated caption
        #     # else:
        #     hidden_state, cell_state = self.decoderRNN.lstm_cell(
        #         word_embed, (hidden_state, cell_state))

        #     # output of the attention mechanism
        #     out = self.decoderRNN.fc_out(hidden_state)
        #     outputs[:, t, :] = out
        #     #print(f"out shape:{out.shape}")
        #     captions.append(torch.argmax(out, dim=1))
        #     #print(f"\n predicted outputs:{captions}")
        #     word_embed = self.decoderRNN.embed(
        #         torch.argmax(out[0])).unsqueeze(0)
        #     last_word_idx = captions[-1].item()
        #     if vocab.itos[last_word_idx] == vocab.stoi["<EOS>"]:
        #         print("BREAKING!!!!!!")
        #         break

        #         # build the output tensor
        #     # print(captions)
        #     # covert the vocab idx to words and return sentence
        # print(f"outputs shape:{outputs.shape}")
        # print(f"Outputs argmax dim2:{torch.argmax(outputs,dim=2)}")
        # return [vocab.itos[idx.item()] for idx in captions if idx.item() != vocab.stoi["<PAD>"]]
        features = self.encoderCNN(image)
        decoded = self.decoderRNN.sample(features, max_len)
        return decoded


# Training

## training function

In [59]:
import torch.optim as optim
from tqdm import tqdm

def show_image(img, title=None, transform=True, f_name=""):
    """Imshow for Tensor."""
    # unnormalize
    if transform:
        img[0] = img[0] * 0.229
        img[1] = img[1] * 0.224
        img[2] = img[2] * 0.225
        img[0] += 0.485
        img[1] += 0.456
        img[2] += 0.406

    img = img.numpy().transpose((1, 2, 0))

    # title = title.replace("<SOS>","").replace("<EOS>", "")
    if title is not None:
        plt.title(title)
    plt.imsave(f'{f_name.replace(".png", "")}_{title}.png', img)
    print(f'Saved {f_name} with caption {plt.title}')
    plt.pause(0.001)  # pause a bit so that plots are updated
    


def train(max_epochs: int, model, progress=250):
    """
    Train a given model
    Args:
        max_epochs (int): Number of epoches to train on
        model ([type]): Model to train
        data_loader ([type]): Dataloader
        device (str): CPU or GPU
        progress (int, optional): Show prediction and loss values every X iterations. Defaults to 250.

    Returns:
        [type]: Trained model
    """
    # Hyperparameters
    learning_rate = 3e-4
    # init model
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()

    # start epochs
    for epoch in range(max_epochs):
        for idx, (img, captions, length) in tqdm(
            enumerate(data_loader), total=len(data_loader), leave=False
        ):
            img = img.to(device)
            captions = captions.to(device).long()
            output = model(img, captions, length)
            loss = criterion(
                output.reshape(-1, output.shape[2]), captions.reshape(-1))
            optimizer.zero_grad()
            loss.backward(loss)
            optimizer.step()
            if idx > 0 and idx % 100 == 0:
                torch.save({'model_state_dict': model.state_dict()}, "checkpoint.torch")
                            
                dataiter = iter(data_loader)
                img_show, cap, cap_len = next(dataiter)
                output = model(img_show.to(device),
                               cap.to(device).long(), cap_len).to(device)
                print(f"\n\nLoss {loss.item():.5f}\n")
                print(f"\nForward\n")
                out_cap = torch.argmax(output[0], dim=1)
                demo_cap = ' '.join([data_loader.dataset.vocab.itos[idx2.item(
                )] for idx2 in out_cap if idx2.item() != data_loader.dataset.vocab.stoi["<PAD>"]])
                # show_image(show_img[0], title=demo_cap, f_name="Forward.png")
                print(demo_cap)
                demo_cap = model.caption_images(img_show[0:1].to(
                    device), vocab=data_loader.dataset.vocab, max_len=30)
                demo_cap = ' '.join(demo_cap)
                print("Predicted")
                print(demo_cap)
                # show_image(img_show[0], title=demo_cap, f_name="Predicted.png")
                print("Original")
                cap = cap[0]
                # print(cap.long())
                demo_cap = ' '.join([data_loader.dataset.vocab.itos[idx2.item(
                )] for idx2 in cap if idx2.item() != data_loader.dataset.vocab.stoi["<PAD>"]])
                print(demo_cap)
                # show_image(img_show[0], title=demo_cap, transform=False, f_name="Original.png")
    return model

## image function

In [51]:
import matplotlib.pyplot as plt
def show_image(img, title=None, transform=True, f_name=''):
    """Imshow for Tensor."""
    
    #unnormalize 
    if transform:
      img[0] = img[0] * 0.229
      img[1] = img[1] * 0.224 
      img[2] = img[2] * 0.225 
      img[0] += 0.485 
      img[1] += 0.456 
      img[2] += 0.406
      
    img = img.numpy().transpose((1, 2, 0))
    
    
    plt.imshow(img)
    # title = title.replace("<SOS>","").replace("<EOS>", "")
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

## Overfit sanity check

In [52]:
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)
def overfit(model, T=250):
    """
    Run a training on one image+caption
    Args:
        model ([type]): Model to train
        device ([type]): CPU or GPU
        data_loader ([type]): Dataloader
        T (int, optional): How many iterations to run training for. Defaults to 250.
    """
    tqdm_bar = partial(tqdm, position=0, leave=True)

    learning_rate = 3e-4


    # init model
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()


    dataiter = iter(data_loader)
    img, caption, length = next(dataiter)
    for i in tqdm_bar(range(T)):
        # train on the same image and caption to achieve overfitting
        img = img.to(device)
        caption = caption.to(device).long()
        output = model(img, caption, length).to(device)
        loss = criterion(
            output.reshape(-1, output.shape[2]), caption.reshape(-1))
        optimizer.zero_grad()
        loss.backward(loss)
        optimizer.step()

    output = model(img, caption, length).to(device)
    show_img = img.to("cpu")
    print(f"\n\nLoss {loss.item():.5f}\n")
    out_cap = torch.argmax(output[0], dim=1)
    demo_cap = ' '.join([data_loader.dataset.vocab.itos[idx2.item(
    )] for idx2 in out_cap if idx2.item() != data_loader.dataset.vocab.stoi["<PAD>"]])
    show_image(show_img[0], title=demo_cap, f_name="Forward.png")
    print("Predicted")
    with torch.no_grad():
        model.eval()
        demo_cap = model.caption_images(show_img[0:1].to(
            device), vocab=data_loader.dataset.vocab, max_len=15)
        demo_cap = ' '.join(demo_cap)
        model.train()

        show_image(show_img[0], title=demo_cap,
                   transform=False, f_name="Predicted.png")
    print("Original")
    cap = caption[0]
    # print(cap.long())
    demo_cap = ' '.join([data_loader.dataset.vocab.itos[idx2.item(
    )] for idx2 in cap if idx2.item() != data_loader.dataset.vocab.stoi["<PAD>"]])
    show_image(show_img[0], title=demo_cap,
               transform=False, f_name="Original.png")

## Process

In [61]:
embed_size = 256
hidden_size = 256
vocab_size = len(dataset.vocab)
model = CNNtoRNN(2304, embed_size, hidden_size, vocab_size, train_CNN=False)
trained_model = train(3, model)




Loss 5.55838


Forward

<SOS> a a a a a a <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>


RuntimeError: input.size(-1) must be equal to input_size. Expected 2560, got 2304

# Misc

In [ ]:
embed_size = 256
hidden_size = 256
vocab_size = len(dataset.vocab)
model = CNNtoRNN(2048,embed_size, hidden_size, vocab_size, train_CNN=False)
overfit(model, 500)
del model

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
for imgs,caps in data_loader:
    print(f"\nimg shape:{imgs.shape}")
    print(f"\ncap shape:{caps.shape}")
    print(f"\nFirst caption:{caps[0]}")
    print(f"\nFirst embed:{caps[0][0]}")
    break

In [ ]:
!conda install -c conda-forge -y ipywidgets 

In [ ]:
!source /home/yandex/DLW2021/davidhay/anaconda3/bin/activate
!conda info

In [ ]:
!conda activate /home/yandex/DLW2021/davidhay/anaconda3/envs/new-env
!conda info --envs




In [16]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()


0

In [3]:
import gc
import torch
def memReport():
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            print(type(obj), obj.size())
gc.collect()
memReport()

/home/yandex/DLW2021/davidhay/anaconda3/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:151: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


In [2]:
!nvidia-smi
!kill -9 40485

Sat Aug 28 11:43:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 460.27.04    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN Xp            On   | 00000000:3D:00.0 Off |                  N/A |
| 33%   55C    P2   119W / 250W |   5061MiB / 12196MiB |     70%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            On   | 00000000:3E:00.0 Off |                  N/A |
| 23%   

In [ ]:
data_loader.dataset